In [30]:
from typing import Dict, List
from langchain.chat_models import ChatOpenAI
from langchain.prompts import  PromptTemplate, ChatPromptTemplate
from langchain.prompts.few_shot import  FewShotChatMessagePromptTemplate, FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.example_selector import LengthBasedExampleSelector
from langchain.prompts.example_selector.base import BaseExampleSelector


    

chat = ChatOpenAI(temperature=0.1,
                   streaming=True, 
                   callbacks=[
                      StreamingStdOutCallbackHandler(),
                    ],
                  )

a_template = PromptTemplate.from_template("What is capital of {country}")
b_template = PromptTemplate(
  template="What is capital of {country}",
  input_variables=["country"],
)
a_template.format(country="Austria")
b_template.format(country="Austrailia")
# result = t | chat
# result.invoke({"country":"Austria"})

'What is capital of Austrailia'

In [32]:
examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]

example_prompt = PromptTemplate.from_template(
  "Human:{question} \n AI:{answer}"
)

prompt = FewShotPromptTemplate(
  example_prompt=example_prompt,
  examples=examples,
  input_variables=["country",],
  suffix="Human:What do you know about {country}?"
)

# prompt.format(country="Turkey")
chain = prompt |chat
chain.invoke({
  "country":"Turkey"
})

AI:
        I know this:
        Capital: Ankara
        Language: Turkish
        Food: Kebabs and Baklava
        Currency: Turkish Lira

AIMessageChunk(content='AI:\n        I know this:\n        Capital: Ankara\n        Language: Turkish\n        Food: Kebabs and Baklava\n        Currency: Turkish Lira')

In [ ]:


example_prompt_chat = ChatPromptTemplate.from_messages([
  ("human", "What do you know about {question}?"),
  ("ai", "{answer}")
])

prompt_chat = FewShotChatMessagePromptTemplate(
  example_prompt=example_prompt_chat,
  examples=examples, 
)

final_prompt = ChatPromptTemplate.from_messages(
  [
  ("system","You are a geography expert"),
  prompt_chat,
  ("human", "What do you know about {country}?"),
])

chain = final_prompt |chat
chain.invoke({"country":"Israel"})


In [38]:

class RandomExampleSelector(BaseExampleSelector):
  def __init__(self, examples):
    self.examples = examples

  def add_example(self, example):
    self.examples.append(example)

  def select_examples(self, input_variables) :
    from random import choice
    return [choice(self.examples)]
  
  
example_prompt = PromptTemplate.from_template(
  "Human:{question} \n AI:{answer}"
)

example_selector = LengthBasedExampleSelector(
  examples=examples,
  example_prompt=example_prompt,
  max_length=380,
)

# 내가 만든거
example_selector_my = RandomExampleSelector(  
  examples = examples,
)

prompt = FewShotPromptTemplate(
  example_prompt=example_prompt,
  example_selector=example_selector_my,

  input_variables=["country",],
  suffix="Human:What do you know about {country}?"
)

prompt.format(country="Brazil")

'Human:What do you know about Italy? \n AI:\n        I know this:\n        Capital: Rome\n        Language: Italian\n        Food: Pizza and Pasta\n        Currency: Euro\n        \n\nHuman:What do you know about Brazil?'

In [46]:
from typing import Dict, List
from langchain.chat_models import ChatOpenAI
from langchain.prompts import  PromptTemplate, ChatPromptTemplate
from langchain.prompts.few_shot import  FewShotChatMessagePromptTemplate, FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.example_selector import LengthBasedExampleSelector
from langchain.prompts.example_selector.base import BaseExampleSelector
from langchain.prompts import load_prompt

prompt=load_prompt("./prompt.yaml")

chat = ChatOpenAI(temperature=0.1,
                   streaming=True, 
                   callbacks=[
                      StreamingStdOutCallbackHandler(),
                    ],
                  )
prompt.format(country="Germany")

'What is the capital of Germany'

In [55]:
from langchain.prompts.pipeline import PipelinePromptTemplate



intro= PromptTemplate.from_template(
    """
    You are a role playing assistant.
    And you are impersonating a {character}
"""
)

example= PromptTemplate.from_template(
    """
    This is an example of how you talk:

    Human: {example_question}
    You: {example_answer}
"""
)

start= PromptTemplate.from_template(
    """
    Start now!

    Human: {question}
    You:
"""
)

final= PromptTemplate.from_template(
    """
    {intro}

    {example}

    {start}
"""
)

prompts= [
    ("intro", intro),
    ("example", example),
    ("start", start),
]




full_prompt = PipelinePromptTemplate(final_prompt=final, pipeline_prompts=prompts,)
full_prompt.format(
  character ="Pirate",
  example_question="What is your location?",
  example_answer="Arrrg! That is a secret!!  Arg Arg!!",
  question="What is your food?",
)

SyntaxError: invalid syntax. Perhaps you forgot a comma? (2634485452.py, line 52)

In [50]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate
from langchain.prompts.pipeline import PipelinePromptTemplate

chat= ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

intro= PromptTemplate.from_template(
    """
    You are a role playing assistant.
    And you are impersonating a {character}
"""
)

example= PromptTemplate.from_template(
    """
    This is an example of how you talk:

    Human: {example_question}
    You: {example_answer}
"""
)

start= PromptTemplate.from_template(
    """
    Start now!

    Human: {question}
    You:
"""
)

final= PromptTemplate.from_template(
    """
    {intro}

    {example}

    {start}
"""
)

prompts= [
    ("intro", intro),
    ("example", example),
    ("start", start),
]


full_prompt= PipelinePromptTemplate(
    final_prompt=final,
    pipeline_prompts=prompts,
)


chain= full_prompt| chat

chain.invoke(
    {
        "character": "Pirate",
        "example_question": "What is your location?",
        "example_answer": "Arrrrg! That is a secret!! Arg arg!!",
        "question": "What is your fav food?",
    }
)